In [240]:
import numpy as np
import pywt
import librosa
from PIL import Image
import sys
import soundfile as sf
import cv2

In [241]:
audio_path="demo.wav"
stego_path="stego.wav"
secret_image_path="secret.jpg"
extracted_image_path="extracted.jpg"

DELTA=0.001
DWT_LEVEL=5
WAVELET="db4"
seed_left=40
seed_right=50
THRESHOLD=0.001
DECISION_FLAG=False #pair if true

In [242]:
def image_to_bits(image_path):
  img=Image.open(image_path).convert('RGB')
  arr=np.array(img,dtype=np.uint8)
  img_shape=arr.shape
  flatten_arr=arr.flatten()
  flatten_arr_bits=np.unpackbits(flatten_arr)

  
  
  if(DECISION_FLAG):

    grouped_flattened=np.char.add(flatten_arr_bits[::2].astype(str),flatten_arr_bits[1::2].astype(str))
  else:
    grouped_flattened=flatten_arr_bits
  return grouped_flattened,img_shape

In [243]:
def bits_to_image(bit_array,img_shape):
    bit_array=np.array(bit_array)
    bit_array=np.packbits(bit_array)
    bit_array=bit_array.reshape(-1,3)
    
    return bit_array.reshape(img_shape)

In [244]:
def coeff_modification(coeff,embed_value,delta=DELTA):
    d=0
    if(DECISION_FLAG):
        dither=[i*delta/16 for i in range(1,8,2)]
        d=0
        
        if(embed_value=="00"):
            d= dither[0]
        if(embed_value=="01"):
            d= dither[1]
        if(embed_value=="10"):
            d= dither[2]
        if(embed_value=="11"):
            d= dither[3]
    
    else:
        dither=[i*delta/8 for i in range(1,4,2)]
        d=0
        if(embed_value==0):
            d= dither[0]
        if(embed_value==1):
            d= dither[1]
    return_val=delta*np.round((coeff-d)/delta)+d

    
    return return_val

In [245]:
def dwt_on_audio(audio_path,wavelet=WAVELET,dwt_level=DWT_LEVEL):
    # audio_stereo,sr=librosa.load(audio_path,mono=False)
    audio_stereo,sr=sf.read(audio_path)

    audio_stereo=audio_stereo.T
    audio_left=audio_stereo[0]
    audio_right=audio_stereo[1]
    


    if(audio_left.shape[0]%2!=0):
        audio_left=audio_left[:-1]
        audio_right=audio_right[:-1]

    
    
    
    coeffs_left=pywt.wavedec(audio_left,wavelet,level=dwt_level,mode="periodization")

    coeffs_right=pywt.wavedec(audio_right,wavelet,level=dwt_level,mode="periodization")

    
    return coeffs_left,coeffs_right,sr



In [246]:
def embedding_function(coeffs,embedding_location,embed_array):
    return_coeffs=coeffs
    for i,index in enumerate(embedding_location):
        
        return_coeffs[index]=(coeff_modification(coeffs[index],embed_array[i],DELTA))


    return return_coeffs

In [247]:
def data_split(embedding_data,coeff_len):
    embedding_data=np.array(embedding_data)
    
    cumulative_len=[]
    for i in range(1,len(coeff_len)):
        cumulative_len.append(sum(coeff_len[:i]))
    split_data=np.split(embedding_data,cumulative_len)
    return split_data


In [248]:
image_data,img_shape=image_to_bits(secret_image_path)
print(len(image_data))
coeffs_left,coeffs_right,sr=dwt_on_audio(audio_path,WAVELET,DWT_LEVEL)
d_coeffs_left=(coeffs_left[1:])
d_coeffs_right=(coeffs_right[1:])

level_arr=np.arange(0,DWT_LEVEL)

possible_locations=sum([arr.shape[0] for arr in d_coeffs_left])

payload_len=len(image_data)

coeff_len=[]



if (payload_len>possible_locations*2):
    sys.exit(payload_len)

for arr in d_coeffs_left:
    coeff_len.append(len(arr))


embed_location_left=[]
embed_location_right=[]
embed_proportion=[]
np.random.seed(seed_left)
total_coeff_sum = sum(coeff_len)

def count_valid_coeffs(level_coeffs):
    return sum(1 for coeff in level_coeffs if abs(coeff) > THRESHOLD)

valid_coeffs_left = [count_valid_coeffs(level) for level in d_coeffs_left]
valid_coeffs_right = [count_valid_coeffs(level) for level in d_coeffs_right]

total_valid_left = sum(valid_coeffs_left)
total_valid_right = sum(valid_coeffs_right)

# Check if payload can fit in valid coefficients
if payload_len > (total_valid_left + total_valid_right):
    raise ValueError(f"Payload length ({payload_len}) exceeds available valid embedding locations ({total_valid_left + total_valid_right}).")

# Distribute payload proportionally based on valid coefficients
embed_proportion_left = [int(round((valid_coeffs_left[i] / total_valid_left) * (payload_len / 2))) for i in level_arr]
embed_proportion_right = [int(round((valid_coeffs_right[i] / total_valid_right) * (payload_len / 2))) for i in level_arr]

# Generate embedding locations
embed_location_left = []
embed_location_right = []
np.random.seed(seed_left)

for i in level_arr:
    level_coeffs = d_coeffs_left[i]
    valid_indices = [idx for idx, coeff in enumerate(level_coeffs) if abs(coeff) > THRESHOLD]
    size = embed_proportion_left[i]
    if len(valid_indices) >= size:
        embed_location_left.append(np.random.choice(valid_indices, size=size, replace=False))
    else:
        embed_location_left.append(np.random.choice(valid_indices, size=len(valid_indices), replace=False))

np.random.seed(seed_right)
for i in level_arr:
    level_coeffs = d_coeffs_right[i]
    valid_indices = [idx for idx, coeff in enumerate(level_coeffs) if abs(coeff) > THRESHOLD]
    size = embed_proportion_right[i]
    if len(valid_indices) >= size:
        embed_location_right.append(np.random.choice(valid_indices, size=size, replace=False))
    else:
        embed_location_right.append(np.random.choice(valid_indices, size=len(valid_indices), replace=False))

image_data_left=image_data[:int(len(image_data)/2)]
image_data_right=image_data[int(len(image_data)/2):]


split_data_left=data_split(image_data_left,embed_proportion_left)
split_data_right=data_split(image_data_right,embed_proportion_right)





modified_coeffs_left=[]
modified_coeffs_right=[]

modified_coeffs_left.append(coeffs_left[0])
modified_coeffs_right.append(coeffs_right[0])

for i in level_arr:
    modified_coeffs_left.append(embedding_function(d_coeffs_left[i],embed_location_left[i],split_data_left[i]))
    modified_coeffs_right.append(embedding_function(d_coeffs_right[i],embed_location_right[i],split_data_right[i]))


stego_audio_left=pywt.waverec(modified_coeffs_left,wavelet=WAVELET,mode="periodization")
stego_audio_right=pywt.waverec(modified_coeffs_right,WAVELET,mode="periodization")
max_val=0
max_val = np.max(np.abs(stego_audio_left))
stego_audio_left = stego_audio_left / max_val
stego_audio_right = stego_audio_right / max_val

min_len = min( len(stego_audio_left), len(stego_audio_right))
reconstructed_stereo = np.column_stack((stego_audio_left[:min_len], stego_audio_right[:min_len]))

sf.write(stego_path,reconstructed_stereo,sr)



6558720


In [249]:
def edwt_on_audio(audio_path,wavelet=WAVELET,dwt_level=DWT_LEVEL):
    audio_stereo,sr=sf.read(audio_path)

    audio_stereo=audio_stereo.T
    audio_left=audio_stereo[0]
    audio_right=audio_stereo[1]
    
    audio_left = audio_left * max_val
    audio_right=audio_right*max_val


    if(audio_left.shape[0]%2!=0):
        audio_left=audio_left[:-1]
        audio_right=audio_right[:-1]

    
    
    
    coeffs_left=pywt.wavedec(audio_left,wavelet,level=dwt_level,mode="periodization")

    coeffs_right=pywt.wavedec(audio_right,wavelet,level=dwt_level,mode="periodization")

    
    return coeffs_left,coeffs_right,sr



In [250]:
def extract_coeff(stego_coeff_val):
    dither=[]
    possible_value=[]
    if(DECISION_FLAG):
        dither=[i*DELTA/16 for i in range(1,8,2)]
        possible_value=["00","01","10","11"]
    
    else:
        dither=[i*DELTA/8 for i in range(1,4,2)]
        possible_value=["0","1"]

    
    q=[(DELTA*np.round((stego_coeff_val-d)/DELTA)+d) for d in dither]
    err_val=[abs(stego_coeff_val-q_val) for q_val in q]
    min_index=err_val.index(min(err_val))

    # if err_val[min_index] > DELTA / 2:  
    #     return "00"

    return possible_value[min_index]

    

In [251]:
def extraction_payload(stego_coeff,embedding_location):
    extracted_data=[]

    extracted_data=[extract_coeff(coeff[index]) for idx,coeff in enumerate(stego_coeff) for index in embedding_location[idx]]
    flat_extracted=[]
    if (DECISION_FLAG):
        flat_extracted = [int(digit) for binary in extracted_data for digit in binary]
    else:
        flat_extracted= [int(digit) for digit in extracted_data]
    
    return flat_extracted

        

In [252]:
#extraction part
stego_coeff_left,stego_coeff_right,sr=edwt_on_audio(stego_path,WAVELET,DWT_LEVEL)

stego_d_left=stego_coeff_left[1:]
stego_d_right=stego_coeff_right[1:]

extracted_img_left=[extraction_payload(stego_d_left,embed_location_left)]

extracted_img_right=[extraction_payload(stego_d_right,embed_location_right)]

extracted_img_arr=extracted_img_left+extracted_img_right
extracted_img_arr=np.array(extracted_img_arr)

extracted_img_flattened=extracted_img_arr.flatten()

extracted_image=bits_to_image(extracted_img_flattened,img_shape)


stego_img=Image.fromarray(extracted_image)
stego_img.save(extracted_image_path)


In [ ]:
#to check bit alteration
counter=0
for i in range(len(extracted_img_flattened)):
    if(image_data[i]!=extracted_img_flattened[i]):
        counter=counter+1
print(counter)

